In [1]:
import pandas as pd
import numpy as np

In [2]:
match_best = pd.read_csv('MODEL_DATA/match_best.csv', header=None)
match_second = pd.read_csv('MODEL_DATA/match_second.csv', header=None)
match_third = pd.read_csv('MODEL_DATA/match_third.csv', header=None)
raw_data = pd.read_csv('MODEL_DATA/raw_data.csv', header=None)
Training_data = pd.read_csv('MODEL_DATA/training_data.csv', header=None)
Testing_data = pd.read_csv('MODEL_DATA/testing_data.csv', header=None)

similar01_best = pd.read_csv('MODEL_DATA/similar01_best.csv', header=None)
similar01_second = pd.read_csv('MODEL_DATA/similar01_second.csv', header=None)
similar01_third = pd.read_csv('MODEL_DATA/similar01_third.csv', header=None)

similar02_best = pd.read_csv('MODEL_DATA/similar02_best.csv', header=None)
similar02_second = pd.read_csv('MODEL_DATA/similar02_second.csv', header=None)
similar02_third = pd.read_csv('MODEL_DATA/similar02_third.csv', header=None)

non_match_best = pd.read_csv('MODEL_DATA/non_match_best.csv', header=None)
non_match_second = pd.read_csv('MODEL_DATA/non_match_second.csv', header=None)
non_match_third = pd.read_csv('MODEL_DATA/non_match_third.csv', header=None)

In [3]:
def S(x) :
    if x > -1 and x < 1:
        return x
    elif x >= 1 :
        return 1
    else :
        return -1

In [4]:
U  = similar01_best - similar01_second 
U1 = similar01_second - similar01_third
U2 = similar01_third - similar01_best

U3 = similar02_best - similar01_best 
U4 = similar02_best - non_match_best
# U5 = similar01_best - non_match_best

U = U.append(U1)
U = U.append(U2)
U = U.append(U3)
U = U.append(U4)
# U = U.append(U5)

U = U.reset_index() # 5877 rows × 116 columns
U = U.drop('index', axis=1)

In [5]:
# U

In [6]:
numOfColumns = U.shape[1]

In [7]:
X = Training_data.ix[0:, 0:numOfColumns-1]
Y = Training_data.ix[0:, numOfColumns]
X_Mul_Y = X.mul(Y, axis=0)
Sum_X_Mul_Y = X_Mul_Y.sum()
X_T = X.transpose()
X_T_Dot_X = X_T.dot(X)
Parameter = X_T_Dot_X.as_matrix()

In [8]:
def Optimization(parameter_l, parameter_u, parameter_g, numOfColumns, U) :
    
    # U = match_second - match_third
    data = np.identity( numOfColumns + 1 )
    
    I = pd.DataFrame(data=data[1:,1:])
    C = pd.concat([parameter_l * I, parameter_u * U],ignore_index=True)
    
    # initial w 
    random_num = np.random.uniform(-1, 1, size=numOfColumns)
    w = pd.DataFrame(data=random_num)
    
    C_Dot_W = C.dot(w) / parameter_g
    
    A = C_Dot_W[0].apply(S)
    # C_T : C 's transpose
    C_T = C.transpose()
    C_T_Dot_A = 0.5 * C_T.dot(A)
    
    Target = Sum_X_Mul_Y - C_T_Dot_A
    Target = Target.as_matrix()
    
    w = np.linalg.solve(Parameter, Target)
    
    iterations = 0
    delta_A = 1000
    delta_w = 1000
    bound = 0.001

    while delta_A > bound and delta_w > bound :
        iterations += 1
        if iterations > 30 :
            return None
        
        C_Dot_W = C.dot(w) / parameter_g

        # update vector A
        A_now = C_Dot_W.apply(S)

        C_T_Dot_A = 0.5 * C_T.dot(A_now)
        Target = Sum_X_Mul_Y - C_T_Dot_A
        Target = Target.as_matrix()

        # update vector w
        w_now = np.linalg.solve(Parameter, Target)

        delta_A = np.linalg.norm(A_now-A)
        delta_w = np.linalg.norm(w_now-w)

        A = A_now
        w = w_now

        print 'iterations:' + str(iterations) + ' ' + \
                'delta_A:' + str(delta_A) + ' ' + \
                'delta_w:' + str(delta_w)  
    
    return w      

In [9]:
def Evaluation(w) :
    
    result = np.dot(test_features, w)
    predict_flag = result > 0
    
    true_positive = 0.0
    true_negative = 0.0
    false_positive = 0.0
    false_negative = 0.0

    for i in range( len(predict_flag) ):
        if predict_flag[i] == True and test_flag[i] == 1 :
            true_positive += 1
        if predict_flag[i] == False and test_flag[i] == -1 :
            true_negative += 1
        if predict_flag[i] == True and test_flag[i] == -1 :
            false_positive += 1
        if predict_flag[i] == False and test_flag[i] == 1 :
            false_negative += 1
            
    ACC = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    PRE = true_positive / (true_positive + false_positive)
    REC = true_positive / (true_positive + false_negative)
    F1 = 2*( PRE * REC) / (PRE + REC)
    
    print ACC,PRE,REC,F1
    
    return F1 # ACC , PRE, REC

In [10]:
test_features = Testing_data.ix[0:,  0:numOfColumns-1]
test_flag = Testing_data.ix[0:, numOfColumns]

In [11]:
w = Optimization(0.01,1.07,0.23, numOfColumns, U)

iterations:1 delta_A:287.280024138 delta_w:0.0955786849663
iterations:2 delta_A:56.8427034025 delta_w:0.0219723973368
iterations:3 delta_A:14.2286271207 delta_w:0.00686881306604
iterations:4 delta_A:3.97488895561 delta_w:0.00223120616172
iterations:5 delta_A:1.20362946172 delta_w:0.000755996734599


In [12]:
F1 = Evaluation(w)

0.794218303338 0.795404185474 0.787327376117 0.791345172484


In [13]:
def writeDict(Dict, filename, sep) :
    with open(filename, 'w') as f:
        for key,value in sorted( Dict.items() ) :
            f.write( str(key) + sep + str(value) + '\n')

In [14]:
def Store_Model( Weights , File ) :
    # Write Model's performance to the File
    File.write(str(F1*100) + '%')
    
    # Write coefficients to the Filse 
    coefficients = Weights
    for i in range( len(coefficients) ) :
        # print i,coefficients[i]
        File.write(',' + str(coefficients[i]) )

In [15]:
with open('./MODEL/MODEL_100.txt', 'w') as model :
    Store_Model(w, model)

In [18]:
parameter_l = 0.01 
parameter_u = 1.13
parameter_g = 0.01
Performace = {}
loops = 0

while parameter_g < 100.0 :
    loops += 1
    print loops 

    if parameter_g < 0.1 :
        parameter_g += 0.01
    elif parameter_g >= 0.1 and parameter_g < 0.5 :
        parameter_g += 0.01
    elif parameter_g >= 0.5 and parameter_g < 1.0 :
        parameter_g += 0.1
    elif parameter_g >= 1.0 and parameter_g < 2.0 :
        parameter_g += 0.1
    elif parameter_g >= 2.0 and parameter_g < 10.0 :
        parameter_g += 1.0
    else :
        parameter_g += 10.0

    key = str(parameter_l) + ',' + str(parameter_u) + str(',') + str(parameter_g)
    # print key
    w = Optimization(parameter_l, parameter_u, parameter_g, numOfColumns, U)
    # print Evaluation(w)
    if w is not None :
        Performace[key] =  Evaluation(w) 

writeDict(Performace, './Tuning_Parameter_g.csv', ':')

1
iterations:1 delta_A:306.841833283 delta_w:0.0882463061428
iterations:2 delta_A:113.53117504 delta_w:0.0219957052238
iterations:3 delta_A:50.2742987959 delta_w:0.00807480881988
iterations:4 delta_A:20.6855184013 delta_w:0.0032903390498
iterations:5 delta_A:8.87057284235 delta_w:0.00153578744947
iterations:6 delta_A:4.00114336771 delta_w:0.000721284643223
0.79401699078 0.795126353791 0.787246141348 0.791166625847
2
iterations:1 delta_A:302.885460294 delta_w:0.0985453918819
iterations:2 delta_A:112.181418735 delta_w:0.023681444725
iterations:3 delta_A:45.1965734884 delta_w:0.00729234355253
iterations:4 delta_A:16.0879499669 delta_w:0.00243665032418
iterations:5 delta_A:5.73346386591 delta_w:0.000862757813041
0.793896203245 0.795027488307 0.787083671812 0.791035637017
3
iterations:1 delta_A:311.860935618 delta_w:0.0919088500039
iterations:2 delta_A:107.008960338 delta_w:0.0221960204812
iterations:3 delta_A:40.4361547909 delta_w:0.00831446541602
iterations:4 delta_A:14.7155293996 delta_w

In [17]:
parameter_l = 0.01 
parameter_u = 0.01
parameter_g = 0.22
Performace = {}
loops = 0

while parameter_u < 100.0 :
    loops += 1
    print loops 

    if parameter_u < 0.1 :
        parameter_u += 0.01
    elif parameter_u >= 0.1 and parameter_u < 1.0 :
        parameter_u += 0.1
    elif parameter_u >= 1.0 and parameter_u < 1.5 :
        parameter_u += 0.01
    elif parameter_u >= 1.5 and parameter_u < 10 :
        parameter_u += 1.0
    else :
        parameter_u += 10.0

    key = str(parameter_l) + ',' + str(parameter_u) + str(',') + str(parameter_g)
    # print key
    w = Optimization(parameter_l, parameter_u, parameter_g, numOfColumns, U)
    # print Evaluation(w)
    if w is not None :
        Performace[key] =  Evaluation(w) 

writeDict(Performace, './Tuning_Parameter_u.csv', ':')

1
iterations:1 delta_A:113.648186623 delta_w:0.000846497085148
0.793131215525 0.793789939374 0.787083671812 0.790422581171
2
iterations:1 delta_A:138.865934881 delta_w:0.00165328974439
iterations:2 delta_A:0.0495346618658 delta_w:7.58086753472e-07
0.793131215525 0.793789939374 0.787083671812 0.790422581171
3
iterations:1 delta_A:153.572188483 delta_w:0.00220573521649
iterations:2 delta_A:0.0960081732821 delta_w:1.636342764e-06
0.793131215525 0.793789939374 0.787083671812 0.790422581171
4
iterations:1 delta_A:168.78680572 delta_w:0.00240452375647
iterations:2 delta_A:0.152377062139 delta_w:2.68892802911e-06
0.793090953014 0.793724911936 0.787083671812 0.790390341396
5
iterations:1 delta_A:174.273842273 delta_w:0.00336724762396
iterations:2 delta_A:0.224280079833 delta_w:5.38347737184e-06
0.793090953014 0.793724911936 0.787083671812 0.790390341396
6
iterations:1 delta_A:175.585773298 delta_w:0.00393129008522
iterations:2 delta_A:0.303936099119 delta_w:8.74573719401e-06
0.793050690502 0.7